<a href="https://colab.research.google.com/github/tamandakaunda-15/chatbot_summative_assingment_agribot/blob/main/Agribot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Exploration and Preprocession


Data set used is a Malawi Secondary Ceritificate of Examination syllabus for the Agriculture subject subject, with close to 200 questions. The set of questions are a summary from over 5 MSCE syllabus books that the Ministry of Education in Malawi approved for the Agriculture subject. This chatbot will help secondary school students prepare for Agriculture examinations and increase the passing of of students taking the Agriculture subject.


In [5]:
!sudo apt-get install tesseract-ocr -y
!sudo apt-get install poppler-utils -y
%pip install PyMuPDF

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.11).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import fitz
import re
import pandas as pd
import json
from datasets import Dataset

# Define the file path for your new agriculture PDF
pdf_file_path = "/content/drive/MyDrive/summative_assignment_chatbot/AGRICULTURE-BOOK-4-Q-A.pdf"

# Function to extract text from a digital PDF
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    doc.close()
    return full_text

# Run the function to get the text
raw_text = extract_text_from_pdf(pdf_file_path)

# Print a small portion of the raw text to verify the extraction
print(raw_text[:1000])

# Check the length to see if any text was extracted
print(f"\nTotal characters extracted: {len(raw_text)}")

 
 
Compiled by L.L Mpandason  
1 
MSCE AGRICULTURE 
 
QUESTIONS  
& 
 MODEL 
ANSWER 
 
MPANDASON 
 
 
 
Compiled by L.L Mpandason  
2 
 
LUKE LINGISON MPANDASON 
P. O Box 32 
Phalula  
Balaka  
Cell: 0881 615 761/ 0995 516 420 
Email:  lingisonluke08@gmail.com/ lukempandason@gmail.com 
 
 
Under reference from Longhorn Publishers (greymattermw.com)  
 
 
 
©Luke L. Mpandason  
 
 
 
All rights reserved. No part of this publication may be reproduced, stored in a retrieval system or transmitted in any 
form or by any means, electronic, mechanical, photocopying or otherwise without prior written permission of the 
copyright owner 
 
 
 
First published, 2020 
 
 
ISBN 978 9996 038 617 
 
 
 
 
Compiled by L.L Mpandason  
3 
Topics covered  
1. Agricultural and the environment 
 Soil degradation 
 Agricultural and climate change 
 Land drainage  
2. Agricultural research and technology  
 Farm mechanization  
 Farm power  
 Gender and agricultural technology  
 Improved farming tec

In [8]:
def parse_agriculture_qa(text):
    # Split the text by the question number pattern
    chunks = re.split(r'(\d+\.\s)', text)[1:]

    qa_pairs = []

    # Iterate through the chunks to form Q&A pairs
    for i in range(0, len(chunks), 2):
        question_number = chunks[i].strip()
        content = chunks[i+1].strip()

        # The answer starts on a new line or with a bullet point
        lines = content.split('\n', 1)
        if len(lines) > 1:
            question_text = lines[0].strip()
            answer_text = lines[1].strip()

            qa_pairs.append({
                "question": question_text,
                "context": f"{question_text}\n{answer_text}",
                "answer": answer_text
            })
        else:
            print(f"Warning: Failed to parse Q&A for chunk starting with {question_number}")

    return qa_pairs

# Run the parsing function on your raw text
qa_dataset = parse_agriculture_qa(raw_text)

# Save to a JSON file
qa_df = pd.DataFrame(qa_dataset)
qa_df.to_json('msce_agriculture_qa.json', orient='records', indent=4)

print(f"\nSuccessfully parsed {len(qa_dataset)} Q&A pairs.")
print("\nDataset saved to 'msce_agriculture_qa.json'.")


Successfully parsed 161 Q&A pairs.

Dataset saved to 'msce_agriculture_qa.json'.


In [9]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import DistilBertTokenizerFast

# Load your structured data from the JSON file you just created
qa_df = pd.read_json('msce_agriculture_qa.json')

# Convert the pandas DataFrame to a Hugging Face Dataset object
dataset = Dataset.from_pandas(qa_df)

# Initialize the tokenizer for DistilBERT
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert/distilbert-base-cased-distilled-squad')

# Display the first example to check the format
print(dataset[0])

{'question': 'Agricultural and the environment', 'context': 'Agricultural and the environment\n\uf0b7 Soil degradation \n\uf0b7 Agricultural and climate change \n\uf0b7 Land drainage', 'answer': '\uf0b7 Soil degradation \n\uf0b7 Agricultural and climate change \n\uf0b7 Land drainage'}


In [10]:
def prepare_train_features(examples):
    # Tokenize the questions and contexts with truncation and padding
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # Truncate the context if it's too long
        max_length=512,
        stride=128, # Add stride for overlapping chunks
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Map each tokenized example to its original example and find answer positions
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to the context and find the start and end of the context
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_sequence_index = 1 # This is typically 1 for the second sequence (context)

        # Find the start and end token indices of the context
        start_token_index = sequence_ids.index(context_sequence_index) if context_sequence_index in sequence_ids else None
        end_token_index = len(sequence_ids) - 1
        while end_token_index > 0 and sequence_ids[end_token_index] != context_sequence_index:
            end_token_index -= 1
        if end_token_index == 0 and sequence_ids[end_token_index] != context_sequence_index:
             end_token_index = None


        # If the answer is not in the context, set the cls_index as the answer.
        sample_index = sample_mapping[i]
        answer = examples["answer"][sample_index]
        context = examples["context"][sample_index]

        start_char = context.find(answer)
        end_char = start_char + len(answer)

        if start_token_index is None or end_token_index is None or start_char < 0 or end_char > len(context):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Otherwise, find the start and end token index of the answer in the context.
            tokenized_examples["start_positions"].append(start_token_index)
            while tokenized_examples["start_positions"][-1] < len(offsets) and offsets[tokenized_examples["start_positions"][-1]][0] <= start_char:
                tokenized_examples["start_positions"][-1] += 1

            tokenized_examples["end_positions"].append(end_token_index)
            while tokenized_examples["end_positions"][-1] >= start_token_index and offsets[tokenized_examples["end_positions"][-1]][1] >= end_char:
                tokenized_examples["end_positions"][-1] -= 1

            # Correct the start and end indices
            tokenized_examples["start_positions"][-1] -= 1
            tokenized_examples["end_positions"][-1] += 1


            # If the tokenized answer is out of the span limits, set the index to the CLS token
            if not (tokenized_examples["start_positions"][-1] >= start_token_index and tokenized_examples["end_positions"][-1] <= end_token_index):
                 tokenized_examples["start_positions"][-1] = cls_index
                 tokenized_examples["end_positions"][-1] = cls_index

    return tokenized_examples

# Apply the function to the entire dataset
tokenized_dataset = dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=dataset.column_names
)

Map:   0%|          | 0/161 [00:00<?, ? examples/s]

In [11]:
!pip install wandb -qU

In [12]:
import wandb
import random
import math

In [ ]:
from transformers import DistilBertForQuestionAnswering, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np

# Load the pre-trained DistilBERT model
model = DistilBertForQuestionAnswering.from_pretrained('distilbert/distilbert-base-cased-distilled-squad')

# Define the PEFT (LoRA) configuration for efficient fine-tuning
peft_config = LoraConfig(
    task_type=TaskType.QUESTION_ANS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./qa_model_output",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
    report_to="wandb"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Start training!
trainer.train()

# Save the fine-tuned model
trainer.save_model("msce_agriculture_chatbot_model")

trainable params: 296,450 || all params: 65,488,900 || trainable%: 0.4527


/tmp/ipython-input-2214310517.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: t-kaunda (t-kaunda-african-leadership-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,No log,4.516135
2,No log,4.201675
3,No log,3.925224
4,No log,3.694288


In [ ]:
# Create a path in your Google Drive to save the model
save_path = '/content/drive/MyDrive/summative_assignment_chatbot/msce_agriculture_chatbot_model'

# Save the fine-tuned model to Google Drive
trainer.save_model(save_path)